In [ ]:
# Download data from Kaggle
# !kaggle datasets download -d yuanyuwendymu/airline-delay-and-cancellation-data-2009-2018

In [3]:
# imports
import pandas as pd
import zipfile
import os
import requests
import airportsdata
import datetime
from pprint import pprint
from dotenv import dotenv_values

# Extract data for 2018 out of the kaggle dataset
try:
    with zipfile.ZipFile('airline-delay-and-cancellation-data-2009-2018.zip', 'r') as zip_ref:
        zip_ref.extract('2018.csv', '../data/')
except:
    print('Zipped dataset missing from src.')

# Delete zipped dataset from kaggle once extracted
try:
    os.remove('airline-delay-and-cancellation-data-2009-2018.zip')  # Alternatively, you can use os.unlink(file_path)
except OSError as e:
    print(f"Error while deleting: {e}")

# import variables
airports = airportsdata.load('IATA')
weather_key = dotenv_values().get('weather_key')

Zipped dataset missing from src.
Error while deleting: [WinError 2] The system cannot find the file specified: 'airline-delay-and-cancellation-data-2009-2018.zip'


# Airline ETL

In [3]:
# read in CSV
df = pd.read_csv('../data/2018.csv')
# display
df.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,...,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,...,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,...,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,UA,2426,SNA,DEN,1335,1330.0,-5.0,15.0,1345.0,...,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2018-01-01,UA,2425,RSW,ORD,1546,1552.0,6.0,19.0,1611.0,...,190.0,182.0,157.0,1120.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2018-01-01,UA,2424,ORD,ALB,630,650.0,20.0,13.0,703.0,...,112.0,106.0,83.0,723.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.count()

FL_DATE                7213446
OP_CARRIER             7213446
OP_CARRIER_FL_NUM      7213446
ORIGIN                 7213446
DEST                   7213446
CRS_DEP_TIME           7213446
DEP_TIME               7101129
DEP_DELAY              7096212
TAXI_OUT               7097616
WHEELS_OFF             7097617
WHEELS_ON              7094200
TAXI_IN                7094200
CRS_ARR_TIME           7213446
ARR_TIME               7094201
ARR_DELAY              7076406
CANCELLED              7213446
CANCELLATION_CODE       116584
DIVERTED               7213446
CRS_ELAPSED_TIME       7213436
ACTUAL_ELAPSED_TIME    7079004
AIR_TIME               7079004
DISTANCE               7213446
CARRIER_DELAY          1352710
WEATHER_DELAY          1352710
NAS_DELAY              1352710
SECURITY_DELAY         1352710
LATE_AIRCRAFT_DELAY    1352710
Unnamed: 27                  0
dtype: int64

In [5]:
# Started Cleaning
airline_df = df.copy()
airline_df = airline_df[['FL_DATE','ORIGIN', 'DEST', 'OP_CARRIER_FL_NUM', 'OP_CARRIER', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY','CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY']]
airline_df.head()

,FL_DATE,ORIGIN,DEST,OP_CARRIER_FL_NUM,OP_CARRIER,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_DEP_TIME,DEP_TIME,DEP_DELAY
0,2018-01-01,EWR,DEN,2429,UA,1745,1722.0,-23.0,1517,1512.0,-5.0
1,2018-01-01,LAS,SFO,2427,UA,1254,1230.0,-24.0,1115,1107.0,-8.0
2,2018-01-01,SNA,DEN,2426,UA,1649,1636.0,-13.0,1335,1330.0,-5.0
3,2018-01-01,RSW,ORD,2425,UA,1756,1754.0,-2.0,1546,1552.0,6.0
4,2018-01-01,ORD,ALB,2424,UA,922,936.0,14.0,630,650.0,20.0


In [6]:
airline_df.rename(columns={
    'FL_DATE': 'Flight_Date',
    'ORIGIN': 'Origin_Airport',
    'DEST': 'Destination_Airport',
    'OP_CARRIER_FL_NUM': 'Flight_Number',
    'OP_CARRIER': 'Operating_Carrier',
    'CRS_ARR_TIME': 'Scheduled_Arrival_Time',
    'ARR_TIME': 'Actual_Arrival_Time',
    'ARR_DELAY': 'Arrival_Delay',
    'CRS_DEP_TIME': 'Scheduled_Departure_Time',
    'DEP_TIME': 'Actual_Departure_Time',
    'DEP_DELAY': 'Departure_Delay'
}, inplace=True)

In [7]:
airline_df.dropna(inplace=True)
airline_df.reset_index(drop=True, inplace=True)
airline_df.head()

,Flight_Date,Origin_Airport,Destination_Airport,Flight_Number,Operating_Carrier,Scheduled_Arrival_Time,Actual_Arrival_Time,Arrival_Delay,Scheduled_Departure_Time,Actual_Departure_Time,Departure_Delay
0,2018-01-01,EWR,DEN,2429,UA,1745,1722.0,-23.0,1517,1512.0,-5.0
1,2018-01-01,LAS,SFO,2427,UA,1254,1230.0,-24.0,1115,1107.0,-8.0
2,2018-01-01,SNA,DEN,2426,UA,1649,1636.0,-13.0,1335,1330.0,-5.0
3,2018-01-01,RSW,ORD,2425,UA,1756,1754.0,-2.0,1546,1552.0,6.0
4,2018-01-01,ORD,ALB,2424,UA,922,936.0,14.0,630,650.0,20.0


In [8]:
airline_df.count()

Flight_Date                 7071818
Origin_Airport              7071818
Destination_Airport         7071818
Flight_Number               7071818
Operating_Carrier           7071818
Scheduled_Arrival_Time      7071818
Actual_Arrival_Time         7071818
Arrival_Delay               7071818
Scheduled_Departure_Time    7071818
Actual_Departure_Time       7071818
Departure_Delay             7071818
dtype: int64

In [9]:
airline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7071818 entries, 0 to 7071817
Data columns (total 11 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   Flight_Date               object 
 1   Origin_Airport            object 
 2   Destination_Airport       object 
 3   Flight_Number             int64  
 4   Operating_Carrier         object 
 5   Scheduled_Arrival_Time    int64  
 6   Actual_Arrival_Time       float64
 7   Arrival_Delay             float64
 8   Scheduled_Departure_Time  int64  
 9   Actual_Departure_Time     float64
 10  Departure_Delay           float64
dtypes: float64(4), int64(3), object(4)
memory usage: 593.5+ MB


In [10]:
len(airline_df['Flight_Date'].unique())

365

In [11]:
airline_df['Flight_Date'].value_counts()

2018-07-13    21797
2018-07-19    21744
2018-07-26    21650
2018-07-20    21646
2018-08-06    21618
              ...  
2018-01-20    14788
2018-02-03    14623
2018-02-10    14568
2018-01-13    14462
2018-11-22    12260
Name: Flight_Date, Length: 365, dtype: int64

In [12]:
# get month with most flights
monthly_df = airline_df.copy()

monthly_df['Flight_DateTime'] = pd.to_datetime(monthly_df['Flight_Date'])
monthly_df['Month'] = monthly_df['Flight_DateTime'].dt.month
month_counts = monthly_df['Month'].value_counts()
most_frequent_month = month_counts.idxmax()
month_most_flights = monthly_df[monthly_df['Month'] == most_frequent_month]
month_most_flights.drop(['Month', 'Flight_DateTime'], axis=1, inplace=True)
month_most_flights.reset_index(drop=True, inplace=True)
month_most_flights.head()


C:\Users\benny\AppData\Local\Temp\ipykernel_10804\3110850706.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_most_flights.drop(['Month', 'Flight_DateTime'], axis=1, inplace=True)


,Flight_Date,Origin_Airport,Destination_Airport,Flight_Number,Operating_Carrier,Scheduled_Arrival_Time,Actual_Arrival_Time,Arrival_Delay,Scheduled_Departure_Time,Actual_Departure_Time,Departure_Delay
0,2018-07-01,ATL,GTR,3280,9E,1540,1518.0,-22.0,1522,1519.0,-3.0
1,2018-07-01,GTR,ATL,3280,9E,1825,1805.0,-20.0,1605,1600.0,-5.0
2,2018-07-01,ATL,GTR,3281,9E,1037,1019.0,-18.0,1028,1023.0,-5.0
3,2018-07-01,GTR,ATL,3281,9E,1321,1312.0,-9.0,1105,1056.0,-9.0
4,2018-07-01,ATL,FAY,3282,9E,1505,1451.0,-14.0,1341,1339.0,-2.0


In [13]:
month_most_flights['Flight_Date'].unique()

array(['2018-07-01', '2018-07-02', '2018-07-03', '2018-07-04',
       '2018-07-05', '2018-07-06', '2018-07-07', '2018-07-08',
       '2018-07-09', '2018-07-10', '2018-07-11', '2018-07-12',
       '2018-07-13', '2018-07-14', '2018-07-15', '2018-07-16',
       '2018-07-17', '2018-07-18', '2018-07-19', '2018-07-20',
       '2018-07-21', '2018-07-22', '2018-07-23', '2018-07-24',
       '2018-07-25', '2018-07-26', '2018-07-27', '2018-07-28',
       '2018-07-29', '2018-07-30', '2018-07-31'], dtype=object)

In [17]:
month_most_flights.count()

Flight_Date                 631207
Origin_Airport              631207
Destination_Airport         631207
Flight_Number               631207
Operating_Carrier           631207
Scheduled_Arrival_Time      631207
Actual_Arrival_Time         631207
Arrival_Delay               631207
Scheduled_Departure_Time    631207
Actual_Departure_Time       631207
Departure_Delay             631207
Arrival City                631207
Departure City              631207
dtype: int64

In [23]:
# add airport cities to DF
month_most_flights['Arrival City'] = ''
month_most_flights['Departure City'] = ''
for i, row in month_most_flights.iterrows():
    arr_port, dep_port = row[['Origin_Airport','Destination_Airport']]
    try:
        month_most_flights.at[i, 'Arrival City'] = f"{airports[arr_port]['city']}, {airports[arr_port]['subd']}"
        month_most_flights.at[i, 'Departure City'] = f"{airports[dep_port]['city']}, {airports[dep_port]['subd']}"
    except:
        if arr_port == 'ISN':
            month_most_flights.at[i,'Arrival City'] = 'Williston, North Dakota'
        elif dep_port == 'ISN':
            month_most_flights.at[i, 'Departure City'] = 'Williston, North Dakota'
        else:
            print(f'Index {i} - Airport not found: {arr_port} | {dep_port}')

month_most_flights.head()



C:\Users\benny\AppData\Local\Temp\ipykernel_10804\3456604836.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_most_flights['Arrival City'] = ''
C:\Users\benny\AppData\Local\Temp\ipykernel_10804\3456604836.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  month_most_flights['Departure City'] = ''


,Flight_Date,Origin_Airport,Destination_Airport,Flight_Number,Operating_Carrier,Scheduled_Arrival_Time,Actual_Arrival_Time,Arrival_Delay,Scheduled_Departure_Time,Actual_Departure_Time,Departure_Delay,Arrival City,Departure City
0,2018-07-01,ATL,GTR,3280,9E,1540,1518.0,-22.0,1522,1519.0,-3.0,"Atlanta, Georgia","Columbus/W Point/Starkville, Mississippi"
1,2018-07-01,GTR,ATL,3280,9E,1825,1805.0,-20.0,1605,1600.0,-5.0,"Columbus/W Point/Starkville, Mississippi","Atlanta, Georgia"
2,2018-07-01,ATL,GTR,3281,9E,1037,1019.0,-18.0,1028,1023.0,-5.0,"Atlanta, Georgia","Columbus/W Point/Starkville, Mississippi"
3,2018-07-01,GTR,ATL,3281,9E,1321,1312.0,-9.0,1105,1056.0,-9.0,"Columbus/W Point/Starkville, Mississippi","Atlanta, Georgia"
4,2018-07-01,ATL,FAY,3282,9E,1505,1451.0,-14.0,1341,1339.0,-2.0,"Atlanta, Georgia","Fayetteville, North Carolina"


In [24]:
# clean out data folder of old data
try:
    os.remove('../data/2018.csv')
except OSError as e:
    print(f"Error while deleting: {e}")

# save month data as csv
month_most_flights.to_csv('../data/clean_2018_July.csv', index=False)

# Weather ETL

In [4]:
df = pd.read_csv('../data/clean_2018_July.csv')
weather_columns = ['Origin Cloud Cover', 'Origin Dew', 'Origin Humidity', 'Origin Precipitation', 'Origin Pressure', 'Origin Snow', 'Origin Temperature', 'Origin Visibility', 'Origin Windspeed', 'Origin Precipitation Probability',\
                   'Destination Cloud Cover', 'Destination Dew', 'Destination Humidity', 'Destination Precipitation', 'Destination Pressure', 'Destination Snow', 'Destination Temperature', 'Destination Visibility', 'Destination Windspeed', 'Destination Precipitation Probability']
for column in weather_columns:
    df[column] = ''
df.head()


,Flight_Date,Origin_Airport,Destination_Airport,Flight_Number,Operating_Carrier,Scheduled_Arrival_Time,Actual_Arrival_Time,Arrival_Delay,Scheduled_Departure_Time,Actual_Departure_Time,...,Destination Cloud Cover,Destination Dew,Destination Humidity,Destination Precipitation,Destination Pressure,Destination Snow,Destination Temperature,Destination Visibility,Destination Windspeed,Destination Precipitation Probability
0,2018-07-01,ATL,GTR,3280,9E,1540,1518.0,-22.0,1522,1519.0,...,,,,,,,,,,
1,2018-07-01,GTR,ATL,3280,9E,1825,1805.0,-20.0,1605,1600.0,...,,,,,,,,,,
2,2018-07-01,ATL,GTR,3281,9E,1037,1019.0,-18.0,1028,1023.0,...,,,,,,,,,,
3,2018-07-01,GTR,ATL,3281,9E,1321,1312.0,-9.0,1105,1056.0,...,,,,,,,,,,
4,2018-07-01,ATL,FAY,3282,9E,1505,1451.0,-14.0,1341,1339.0,...,,,,,,,,,,


In [46]:
# group by day and filter rows into dict
final_df = df.copy()
# df['Flight_DateTime'] = pd.to_datetime(df['Flight_Date'])
# flight_day = {}

# for date, group in df.groupby('Flight_DateTime'):
#     flight_day[date.day] = group.copy()

# pprint(flight_day)

In [6]:
# URL params
url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/'
unitGroup = 'unitGroup=us'
contentType = 'contentType=json'

In [19]:
# Function to get weather data
def getWeatherData(weather_resp, flight_time):

    weatherData = {}
    try:
        for time in weather_resp['days'][0]['hours']:

            epoch_time = time['datetimeEpoch']
            dt_obj = datetime.datetime.fromtimestamp(epoch_time)
            military_time_str = dt_obj.strftime('%H%M')
            
            if int(military_time_str) < flight_time:
                weatherData = {
                    'cloudcover': time['cloudcover'],
                    'dew': time['dew'],
                    'humidity': time['humidity'],
                    'precip': time['precip'],
                    'pressure': time['pressure'],
                    'snow': time['snow'],
                    'temp': time['temp'],
                    'visibility': time['visibility'],
                    'windspeed': time['windspeed'],
                    'precipprob': time['precipprob'],
                }
    except:
        print('Error inside getWeatherData function.')
    return weatherData
        

In [48]:

# 2018_07_20
# 2018_07_23
# 2018_07_24
final_df = final_df[(final_df['Flight_Date'] == '2018-07-20') | (final_df['Flight_Date'] == '2018-07-23') | (final_df['Flight_Date'] == '2018-07-24')]
final_df.head()




,Flight_Date,Origin_Airport,Destination_Airport,Flight_Number,Operating_Carrier,Scheduled_Arrival_Time,Actual_Arrival_Time,Arrival_Delay,Scheduled_Departure_Time,Actual_Departure_Time,...,Destination Dew,Destination Humidity,Destination Precipitation,Destination Pressure,Destination Snow,Destination Temperature,Destination Visibility,Destination Windspeed,Destination Precipitation Probability,Flight_DateTime
383827,2018-07-20,ABQ,AUS,5929,WN,1900,1917.0,17.0,1625,1644.0,...,,,,,,,,,,2018-07-20
383828,2018-07-20,ABQ,BWI,2377,WN,2155,2222.0,27.0,1615,1624.0,...,,,,,,,,,,2018-07-20
383829,2018-07-20,ABQ,BWI,2504,WN,1410,1441.0,31.0,825,834.0,...,,,,,,,,,,2018-07-20
383830,2018-07-20,ABQ,DAL,15,WN,905,850.0,-15.0,620,618.0,...,,,,,,,,,,2018-07-20
383831,2018-07-20,ABQ,DAL,942,WN,1620,1622.0,2.0,1340,1339.0,...,,,,,,,,,,2018-07-20


In [20]:
# create function to merge data
def requestWeatherAndMerge(location):
    for index, row in final_df.iterrows():
        flight_date = row['Flight_Date']

        if location.title() == 'Origin':
            city = row['Arrival City']
            fl_time = row['Scheduled_Arrival_Time']
        elif location.title() == 'Destination': 
            city = row['Departure City']
            fl_time = row['Scheduled_Departure_Time']
        # make request for Arrival Data
        try:
            resp = requests.get(f"{url}{city}/{flight_date}/{flight_date}?{unitGroup}&key={weather_key}&{contentType}").json()
        except:
                print(f'Error with API request. {flight_date}')
        weather_data = getWeatherData(resp, fl_time)
        try:
            final_df.at[index, f'{location} Cloud Cover'] = weather_data['cloudcover']
            final_df.at[index, f'{location} Dew'] = weather_data['dew']
            final_df.at[index, f'{location} Humidity'] = weather_data['humidity']
            final_df.at[index, f'{location} Precipitation'] = weather_data['precip']
            final_df.at[index, f'{location} Pressure'] = weather_data['pressure']
            final_df.at[index, f'{location} Snow'] = weather_data['snow']
            final_df.at[index, f'{location} Temperature'] = weather_data['temp']
            final_df.at[index, f'{location} Visibility'] = weather_data['visibility']
            final_df.at[index, f'{location} Windspeed'] = weather_data['windspeed']
            final_df.at[index, f'{location} Precipitation Probability'] = weather_data['precipprob']
        except:
                print('Error merging into final dataframe.')

In [ ]:
requestWeatherAndMerge('Origin')

In [45]:
# final_df[final_df.loc[:, 'Arrival_Delay'] > 0]

,Flight_Date,Origin_Airport,Destination_Airport,Flight_Number,Operating_Carrier,Scheduled_Arrival_Time,Actual_Arrival_Time,Arrival_Delay,Scheduled_Departure_Time,Actual_Departure_Time,...,Destination Cloud Cover,Destination Dew,Destination Humidity,Destination Precipitation,Destination Pressure,Destination Snow,Destination Temperature,Destination Visibility,Destination Windspeed,Destination Precipitation Probability
0,2018-07-01,ATL,GTR,3280,9E,1540,1518.0,-22.0,1522,1519.0,...,,,,,,,,,,
1,2018-07-01,GTR,ATL,3280,9E,1825,1805.0,-20.0,1605,1600.0,...,,,,,,,,,,
2,2018-07-01,ATL,GTR,3281,9E,1037,1019.0,-18.0,1028,1023.0,...,,,,,,,,,,
3,2018-07-01,GTR,ATL,3281,9E,1321,1312.0,-9.0,1105,1056.0,...,,,,,,,,,,
4,2018-07-01,ATL,FAY,3282,9E,1505,1451.0,-14.0,1341,1339.0,...,,,,,,,,,,
